In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
# from src.model import *
# from src.util import *
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 18
BATCH_SIZE = 10
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [5]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [6]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [7]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [8]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)
        
    def forward(self,x):
        embedd_x = self.embedding(x)
        embedd_x.retain_grad()
        x = embedd_x.transpose(-1,-2)
        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()
        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        return x,embedd_x

In [9]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [10]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [11]:
model_embedding_layer = model.embedding

In [12]:
from collections import defaultdict
dic = defaultdict(list)

In [ ]:
total_batch_acc = []
bar = tqdm(testloader)
# bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    optim.zero_grad()
    temp_total_acc = []
    batch_data = batch_data.cuda() if CUDA else batch_data
    clone_batch_data = batch_data.detach().clone()
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp[idx,target] = 1
    temp_label = temp.cuda() if CUDA else temp
    mask = (batch_data == 0).float() # wrong # I forgot what's wrong here
    pred,embedd_x = model(batch_data)
    _, predicted = torch.max(pred, 1)
    acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
    temp_total_acc.append(acc)

    random_padding = torch.randint_like(input=mask,low=1,high=257) * mask
    batch_data += random_padding.long()
    pred,embedd_x = model(batch_data)
    _, predicted = torch.max(pred, 1)
    acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
    temp_total_acc.append(acc)

    for time in range(8):
        pred,embedd_x = model(batch_data)
        pred.mean().backward()
        all_embedd = model.embedding(torch.arange(start=0,end=257).cuda())
        grad = torch.div(
            input = -embedd_x.grad,
            other = torch.linalg.norm(-embedd_x.grad,ord=2,axis=-1).unsqueeze(-1),
        )
        grad = grad.nan_to_num(0)
        shape = embedd_x.shape
        embedd_x = embedd_x.reshape((shape[0],shape[1],1,shape[2]))
        embedd_x  = embedd_x.expand(shape[0],shape[1],257,shape[2])
        shape = grad.shape
        grad = grad.reshape((shape[0],shape[1],1,shape[2]))
        grad = grad.expand(shape[0],shape[1],257,shape[2])
        sb = torch.sum(grad*(all_embedd - embedd_x),dim=-1)
        sb = sb.unsqueeze(-1)
        sb_mask = (sb.squeeze() > 0).float()
        db = torch.linalg.norm(all_embedd - (embedd_x+sb*grad),ord=1,axis=-1)
        # print(torch.any(torch.isnan(db)))
        result = sb_mask*db + (1-sb_mask)*(torch.max(db) + 1)
        result = torch.argmin(result,dim=-1)
        batch_data.data = (batch_data.data*(1-mask) + result*mask).long()
        pred,embedd_x = model(batch_data)
        _, predicted = torch.max(pred, 1)
        acc = (batch_label.cpu().data.numpy() == predicted.cpu().data.numpy()).mean()
        temp_total_acc.append(acc)

    pred = pred.detach().cpu().numpy()
    batch_label = batch_label.detach().cpu().numpy()
    pred = np.argmax(pred,1)
    temp_acc = (batch_label == pred).mean()
    temp = batch_label == pred
    if temp.mean() != 0:
        idxs = np.argwhere(temp).reshape(-1)
        for idx in idxs:
            print(step*BATCH_SIZE+idx)
            padding_num = (clone_batch_data[idx] == 0).sum().tolist()
            print(padding_num)
            dic[padding_num].append(step*BATCH_SIZE+idx)
    total_batch_acc.append(temp_total_acc)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.7f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    bar.set_description(f'{total_batch_acc_str}')

[0.8730769 0.4692308 0.0038462 0.0038462 0.0038462 0.0038462 0.0038462 0.0038462 0.0038462 0.0038462]:   0%|          | 26/10000 [00:12<1:12:45,  2.28it/s]

253
0


[0.8780488 0.4756098 0.0048780 0.0048780 0.0048780 0.0048780 0.0048780 0.0048780 0.0048780 0.0048780]:   0%|          | 41/10000 [00:19<1:12:36,  2.29it/s]

400
0


[0.8600000 0.4538462 0.0046154 0.0046154 0.0046154 0.0046154 0.0046154 0.0046154 0.0046154 0.0046154]:   1%|          | 65/10000 [00:29<1:13:28,  2.25it/s]

...

[0.8560417 0.4414583 0.0059375 0.0058333 0.0058333 0.0058333 0.0058333 0.0058333 0.0058333 0.0058333]:  10%|▉         | 960/10000 [07:08<1:07:13,  2.24it/s]

9595
0


[0.8563017 0.4407025 0.0059917 0.0058884 0.0058884 0.0058884 0.0058884 0.0058884 0.0058884 0.0058884]:  10%|▉         | 968/10000 [07:11<1:07:27,  2.23it/s]

9672
0


[0.8562628 0.4405544 0.0059548 0.0058522 0.0058522 0.0058522 0.0058522 0.0058522 0.0058522 0.0058522]:  10%|▉         | 974/10000 [07:14<1:07:14,  2.24it/s]